In [13]:

import gensim

In [7]:
sentences, cur_sent = list(), list()
with open('./wackypedia_en1.words10.20Mwords') as f:
 for line in f:
    line = line.strip()
    if line == '</s>':
        sentences.append(cur_sent)
        cur_sent = list()
    elif line != '<s>' and not line.startswith('<text') and not line.startswith('</text'):
        cur_sent.append(line.split('\t')[0])

print(sentences)


oard', '-', 'surely', 'you', 'are', 'hinting', 'at', 'that', ',', 'it', 'is', 'impossible', 'to', 'interpret', 'it', 'otherwise', '-', 'is', 'the', 'height', 'of', 'stupidity', '.'], ['If', 'you', 'do', 'not', 'consider', 'me', 'already', 'hopelessly', 'foolish', ',', 'how', 'can', 'you', 'think', 'of', 'that', '?'], ['?'], ['?'], ['?'], ['"', 'From', 'then', 'until', 'his', 'final', 'stroke', ',', 'Lenin', 'spent', 'much', 'of', 'his', 'time', 'trying', 'to', 'devise', 'a', 'way', 'to', 'prevent', 'a', 'split', 'within', 'the', 'Communist', 'Party', 'leadership', ',', 'which', 'was', 'reflected', 'in', 'Lenin', "'s", 'Testament', '.'], ['As', 'part', 'of', 'this', 'effort', ',', 'on', 'September', '11', ',', '1922', 'Lenin', 'proposed', 'that', 'Trotsky', 'become', 'his', 'deputy', 'at', 'the', 'Sovnarkom', '.'], ['The', 'Politburo', 'approved', 'the', 'proposal', ',', 'but', 'Trotsky', '"', 'categorically', 'refused', '"', '.'], ['In', 'late', '1922', ',', 'Lenin', "'s", 'relationshi

In [16]:
model_big_ten = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=10)
model_small_ten = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=10)

model_big_five = gensim.models.Word2Vec(sentences, min_count=5, window=1, vector_size=500)
model_small_five = gensim.models.Word2Vec(sentences, min_count=5, window=10, vector_size=500)

AttributeError: module 'gensim' has no attribute 'Word2Vec'